In [ ]:
import folium
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import datetime
from branca.colormap import linear

# Load GeoDataFrames
north = gpd.read_file('geo_postcode_East.json')
east = gpd.read_file('geo_postcode_Inner.json')
south = gpd.read_file('geo_postcode_North.json')
west = gpd.read_file('geo_postcode_South.json')
inner = gpd.read_file('geo_postcode_West.json')

# Combine and filter GeoDataFrames
postcode = ["3000","3002","3003","3004","3005","3006","3008","3010","3031","3032","3121","3141","3205","3207"]
melb_subdistricts = gpd.GeoDataFrame(pd.concat([north, east, south, west, inner], ignore_index=True))
melb_subdistricts = melb_subdistricts[melb_subdistricts['name'].isin(postcode)]


# Prepare canopy data
merged_canopies = pd.read_csv("Data/Modified/canopies_all.csv")
merged_canopies['geometry'] = merged_canopies['geo_point_2d'].apply(
    lambda x: Point(float(x.split(', ')[1]), float(x.split(', ')[0]))
)
geo_canopies = gpd.GeoDataFrame(merged_canopies, geometry='geometry', crs="EPSG:4326")

# Spatial join
canopies_with_districts = gpd.sjoin(geo_canopies, melb_subdistricts, how="left", op='within')
canopies_with_districts['year'] = canopies_with_districts['year'].astype(int)

# Aggregate data
canopy_area_by_district_year = canopies_with_districts.groupby(['name', 'year'])['shape_area'].sum().reset_index()

# Create colormap
colormap = linear.PuRd_09.scale(canopy_area_by_district_year['shape_area'].min(), canopy_area_by_district_year['shape_area'].max())

# Create timestamps
def create_timestamp(year):
    dt = datetime.datetime(year, 1, 1, tzinfo=datetime.timezone.utc)
    return int(dt.timestamp() * 1000)

# Create styledict
years = canopy_area_by_district_year['year'].unique()
timestamps = {year: create_timestamp(year) for year in years}
styledict1 = {
    str(row['name']): {
        timestamps[row['year']]: {
            'color': colormap(row['shape_area']),
            'opacity': 0.7
        }
    }
    for _, row in canopy_area_by_district_year.iterrows()
}

# Create the map
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)
slider = TimeSliderChoropleth(
    data=canopies_with_districts.to_json(),
    styledict=styledict1
).add_to(map)
colormap.add_to(map)

# Save and display the map
map


In [4]:
import folium
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import datetime
from branca.colormap import linear

# Load GeoDataFrames
north = gpd.read_file('/Users/maqixin/Documents/DTU/Master/Semeter3/Social Data 02806/SocialDataFinalProject/geo_postcode_North.json')
east = gpd.read_file('/Users/maqixin/Documents/DTU/Master/Semeter3/Social Data 02806/SocialDataFinalProject/geo_postcode_East.json')
south = gpd.read_file('/Users/maqixin/Documents/DTU/Master/Semeter3/Social Data 02806/SocialDataFinalProject/geo_postcode_South.json')
west = gpd.read_file('/Users/maqixin/Documents/DTU/Master/Semeter3/Social Data 02806/SocialDataFinalProject/geo_postcode_West.json')
inner = gpd.read_file('/Users/maqixin/Documents/DTU/Master/Semeter3/Social Data 02806/SocialDataFinalProject/geo_postcode_Inner.json')

# Combine and filter GeoDataFrames
postcode = ["3000","3002","3003","3004","3005","3006","3008","3010","3031","3032","3121","3141","3205","3207"]
melb_subdistricts = gpd.GeoDataFrame(pd.concat([north, east, south, west, inner], ignore_index=True))
melb_subdistricts = melb_subdistricts[melb_subdistricts['name'].isin(postcode)]
melb_subdistricts.set_index("name")
print(melb_subdistricts)

     name                                           geometry
62   3121  POLYGON ((144.99135 -37.80973, 144.99198 -37.8...
82   3141  POLYGON ((144.98793 -37.82962, 144.98839 -37.8...
142  3205  POLYGON ((144.94144 -37.82580, 144.94229 -37.8...
144  3207  POLYGON ((144.89836 -37.84120, 144.89812 -37.8...
163  3031  POLYGON ((144.93993 -37.78702, 144.94008 -37.7...
164  3032  POLYGON ((144.87188 -37.77266, 144.87210 -37.7...
176  3000  POLYGON ((144.97136 -37.80773, 144.97168 -37.8...
177  3002  POLYGON ((144.97136 -37.80773, 144.97319 -37.8...
178  3003  POLYGON ((144.91649 -37.79821, 144.91973 -37.7...
179  3004  POLYGON ((144.97041 -37.83016, 144.97132 -37.8...
180  3005  POLYGON ((144.95507 -37.82110, 144.95515 -37.8...
181  3006  POLYGON ((144.94793 -37.82339, 144.94836 -37.8...
182  3008  POLYGON ((144.92037 -37.82003, 144.91989 -37.8...
183  3010  POLYGON ((144.95876 -37.78988, 144.95890 -37.7...


In [ ]:
merged_canopies = pd.read_csv("Data/Modified/canopies_all.csv")
merged_canopies['geometry'] = merged_canopies['geo_point_2d'].apply(
    lambda x: Point(float(x.split(', ')[1]), float(x.split(', ')[0]))
)
geo_canopies = gpd.GeoDataFrame(merged_canopies, geometry='geometry', crs="EPSG:4326")

# Spatial join
canopies_with_districts = gpd.sjoin(geo_canopies, melb_subdistricts, how="left", op='within')
canopies_with_districts['year'] = canopies_with_districts['year'].astype(int)

# Aggregate data
canopy_area_by_district_year = canopies_with_districts.groupby(['name', 'year'])['shape_area'].sum().reset_index()

# Create colormap
colormap = linear.PuRd_09.scale(canopy_area_by_district_year['shape_area'].min(), canopy_area_by_district_year['shape_area'].max())

# Create timestamps
def create_timestamp(year):
    dt = datetime.datetime(year, 1, 1, tzinfo=datetime.timezone.utc)
    return int(dt.timestamp())

# Create styledict
years = canopy_area_by_district_year['year'].unique()
timestamps = {year: create_timestamp(year) for year in years}
styledict1 = {
    str(row['name']): {
        timestamps[row['year']]: {
            'color': colormap(row['shape_area']),
            'opacity': 0.7
        }
    }
    for _, row in canopy_area_by_district_year.iterrows()
}

# Create the map
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)
slider = TimeSliderChoropleth(
    data=canopies_with_districts.to_json(),
    styledict=styledict1
).add_to(map)
colormap.add_to(map)

# Save and display the map
map
